In [10]:
import os
from openpi.shared import download

os.environ["OPENPI_DATA_HOME"] = "/workspace/projects/openpi/jax_checkpoint"   # cache root you want
ckpt_path = download.maybe_download("gs://openpi-assets/checkpoints/pi0_aloha_sim")
print("Downloaded to:", ckpt_path)

  0%|          | 0.00/11.2G [00:00<?, ?iB/s]

Downloaded to: /workspace/projects/openpi/jax_checkpoint/openpi-assets/checkpoints/pi0_aloha_sim


In [1]:
import os

os.environ["MUJOCO_GL"] = "egl"
os.environ["PYOPENGL_PLATFORM"] = "egl"

In [2]:
import sys
print(sys.executable)

/workspace/projects/openpi/.venv/bin/python


In [3]:
# import dataclasses

# import jax

from openpi.models import model as _model
from openpi.policies import aloha_policy
from openpi.training import config as _config
from openpi.training import data_loader as _data_loader

from openpi.training import config as _config
from openpi.policies import policy_config
from openpi.shared import download
from pathlib import Path

In [ ]:
model_id = "pi0_aloha_sim"
config = _config.get_config(model_id)
checkpoint_dir = Path("/workspace/projects/openpi/pytorch_checkpoint/" + model_id)

In [12]:
# Create a trained policy (automatically detects PyTorch format)
policy = policy_config.create_trained_policy(config, checkpoint_dir)
example = aloha_policy.make_aloha_example()
# Run inference (same API as JAX)


In [ ]:
action_chunk = policy.infer(example)["actions"]
action_chunk

## test

In [3]:
import gymnasium as gym
import gym_aloha  # 关键：触发注册（没有这句通常就 NamespaceNotFound）


env = gym.make("gym_aloha/AlohaInsertion-v0", render_mode="rgb_array")

# 👇 就在这里 print
print("env:", env)
print("action_space:", env.action_space)
print("action_space.low:", env.action_space.low)
print("action_space.high:", env.action_space.high)

obs, info = env.reset()
print("type(obs):", type(obs))
if isinstance(obs, dict):
    print("obs keys:", obs.keys())


env: <TimeLimit<OrderEnforcing<PassiveEnvChecker<AlohaEnv<gym_aloha/AlohaInsertion-v0>>>>>
action_space: Box(-1.0, 1.0, (14,), float32)
action_space.low: [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
action_space.high: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
type(obs): <class 'dict'>
obs keys: dict_keys(['top'])


In [4]:
import inspect

print(env.unwrapped)                 # 看真实 env 类
print(env.unwrapped.__class__)       # 类名
print(inspect.getsource(env.unwrapped.step))  # 直接看 step 里怎么用 action


<AlohaEnv<gym_aloha/AlohaInsertion-v0>>
<class 'gym_aloha.env.AlohaEnv'>
    def step(self, action):
        assert action.ndim == 1
        # TODO(rcadene): add info["is_success"] and info["success"] ?

        _, reward, _, raw_obs = self._env.step(action)

        # TODO(rcadene): add an enum
        terminated = is_success = reward == 4

        info = {"is_success": is_success}

        observation = self._format_raw_obs(raw_obs)

        truncated = False
        return observation, reward, terminated, truncated, info



In [6]:
import inspect

inner = env.unwrapped._env
print("inner env:", inner)
print("inner class:", inner.__class__)
print(inspect.getsource(inner.step))


inner env: <dm_control.rl.control.Environment object at 0x72def0f5e790>
inner class: <class 'dm_control.rl.control.Environment'>
  def step(self, action):
    """Updates the environment using the action and returns a `TimeStep`."""

    if self._reset_next_step:
      return self.reset()

    self._task.before_step(action, self._physics)
    self._physics.step(self._n_sub_steps)
    self._task.after_step(self._physics)

    reward = self._task.get_reward(self._physics)
    observation = self._task.get_observation(self._physics)
    if self._flat_observation:
      observation = flatten_observation(observation)

    self._step_count += 1
    if self._step_count >= self._step_limit:
      discount = 1.0
    else:
      discount = self._task.get_termination(self._physics)

    episode_over = discount is not None

    if episode_over:
      self._reset_next_step = True
      return dm_env.TimeStep(
          dm_env.StepType.LAST, reward, discount, observation)
    else:
      return dm_env

In [7]:
import inspect

task = env.unwrapped._env._task
print("task:", task)
print("task class:", task.__class__)

print(inspect.getsource(task.before_step))


task: <gym_aloha.tasks.sim.InsertionTask object at 0x72df05f48c10>
task class: <class 'gym_aloha.tasks.sim.InsertionTask'>
    def before_step(self, action, physics):
        left_arm_action = action[:6]
        right_arm_action = action[7 : 7 + 6]
        normalized_left_gripper_action = action[6]
        normalized_right_gripper_action = action[7 + 6]

        left_gripper_action = unnormalize_puppet_gripper_position(normalized_left_gripper_action)
        right_gripper_action = unnormalize_puppet_gripper_position(normalized_right_gripper_action)

        full_left_gripper_action = [left_gripper_action, -left_gripper_action]
        full_right_gripper_action = [right_gripper_action, -right_gripper_action]

        env_action = np.concatenate(
            [left_arm_action, full_left_gripper_action, right_arm_action, full_right_gripper_action]
        )
        super().before_step(env_action, physics)
        return



## 从服务器获取 pi0 结果并执行

In [6]:
import numpy as np

def make_pi0_example(obs, env, example_template):
    """
    把 ALOHA sim 的真实观测拼成 pi0 期望的 example
    """
    ex = {}

    # 1) 图像
    ex["observation/image"] = obs["top"]
    ex["observation/wrist_image"] = obs["top"]  # 没 wrist cam，先复用

    # 2) dm_control state（关键）
    dm_obs = env.unwrapped._env._task.get_observation(
        env.unwrapped._env._physics
    )

    # 你可以先把 dict flatten 成一个 vector
    state_list = []
    for k, v in dm_obs.items():
        v = np.asarray(v).ravel()
        state_list.append(v)
    state = np.concatenate(state_list).astype(np.float32)

    ex["observation/state"] = state

    # 3) prompt（如果 config 里有）
    if "prompt" in example_template:
        ex["prompt"] = example_template["prompt"]

    return ex


In [7]:
from openpi_client import websocket_client_policy

client = websocket_client_policy.WebsocketClientPolicy(
    host="127.0.0.1",  # 如果 client 和 server 在同一台机器/同一个 pod
    port=8000,
)

# 用你之前能跑通的 dummy example 先测试（最小 sanity check）
from openpi.policies import aloha_policy
example = aloha_policy.make_aloha_example()

out = client.infer(example)
action_chunk = out["actions"]
print(type(action_chunk), getattr(action_chunk, "shape", None))
print(action_chunk[0])


ERROR:websockets.client:keepalive ping failed
TimeoutError: timed out while closing connection

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/workspace/projects/openpi/.venv/lib/python3.11/site-packages/websockets/sync/connection.py", line 777, in keepalive
    with self.send_context():
  File "/usr/lib/python3.11/contextlib.py", line 144, in __exit__
    next(self.gen)
  File "/workspace/projects/openpi/.venv/lib/python3.11/site-packages/websockets/sync/connection.py", line 1012, in send_context
    raise self.protocol.close_exc from original_exc
websockets.exceptions.ConnectionClosedError: sent 1011 (internal error) keepalive ping timeout; no close frame received


ConnectionClosedError: sent 1011 (internal error) keepalive ping timeout; no close frame received

In [ ]:
import numpy as np
import gymnasium as gym
import gym_aloha  # trigger registration
from openpi_client import websocket_client_policy
from openpi.policies import aloha_policy

env = gym.make("gym_aloha/AlohaInsertion-v0", render_mode="rgb_array")
obs, info = env.reset(seed=0)

client = websocket_client_policy.WebsocketClientPolicy(host="127.0.0.1", port=8000)
template = aloha_policy.make_aloha_example()

def build_example(obs, env, template):
    # 这里先用最小版：top 图 + dm_control state（你之后可以再对齐 state 顺序）
    ex = {}
    ex.update(template)

    ex["observation/image"] = obs["top"]
    ex["observation/wrist_image"] = obs["top"]

    dm_obs = env.unwrapped._env._task.get_observation(env.unwrapped._env._physics)
    state = np.concatenate([np.asarray(v).ravel() for v in dm_obs.values()]).astype(np.float32)
    ex["observation/state"] = state

    return ex

H = 10
t = 0

while t < 400:
    ex = build_example(obs, env, template)
    out = client.infer(ex)                 # ✅ action 从 server 来
    chunk = np.asarray(out["actions"], dtype=np.float32)

    for i in range(min(H, len(chunk))):
        a = np.clip(chunk[i], env.action_space.low, env.action_space.high)
        obs, reward, terminated, truncated, info = env.step(a)
        t += 1
        if terminated or truncated:
            print("done", t, reward, info)
            break
    if terminated or truncated:
        break

env.close()


In [13]:
obs, info = env.reset(seed=0)

example_template = aloha_policy.make_aloha_example()

H = 10  # rolling horizon
t = 0

while t < 400:
    example = make_pi0_example(obs, env, example_template)
    out = policy.infer(example)
    action_chunk = np.asarray(out["actions"], dtype=np.float32)

    for i in range(min(H, len(action_chunk))):
        action = action_chunk[i]
        action = np.clip(action,
                         env.action_space.low,
                         env.action_space.high)

        obs, reward, terminated, truncated, info = env.step(action)
        t += 1

        if terminated or truncated:
            print("done", t, reward, info)
            break

    if terminated or truncated:
        break


TypeError: float() argument must be a string or a real number, not 'dict'

In [ ]:
import os
import time
import numpy as np

import gymnasium as gym
import gym_aloha
from gymnasium.wrappers import RecordVideo, TimeLimit

from openpi_client import websocket_client_policy


import numpy as np

In [ ]:
def ensure_hwc_uint8(img: np.ndarray) -> np.ndarray:
    """确保 img 是 HWC, uint8"""
    img = np.asarray(img)
    if img.dtype != np.uint8:
        img = img.astype(np.uint8)

    # 如果是 CHW，就转成 HWC
    if img.ndim == 3 and img.shape[0] in (1, 3) and img.shape[-1] not in (1, 3):
        img = np.transpose(img, (1, 2, 0))

    # 如果是灰度 HW，扩成 HWC
    if img.ndim == 2:
        img = img[:, :, None]

    # 最终必须是 HWC 且 C=1 or 3
    assert img.ndim == 3 and img.shape[-1] in (1, 3), f"bad image shape: {img.shape}"
    return img

def add_batch(img_hwc: np.ndarray) -> np.ndarray:
    """变成 (1,H,W,C)"""
    return img_hwc[None, ...]

In [22]:
def build_aloha_sim_payload(env, gym_obs, prompt: str) -> dict:
    dm_obs = env.unwrapped._env._task.get_observation(env.unwrapped._env._physics)

    # ✅ state：14 维 joint state
    qpos = np.asarray(dm_obs["qpos"], dtype=np.float32).ravel()
    state = qpos[:14]
    if state.shape[0] < 14:
        state = np.pad(state, (0, 14 - state.shape[0]))
    state = state.astype(np.float32)

    imgs = dm_obs.get("images", None)

    top = ensure_hwc_uint8(gym_obs["top"])
    top = top

    def pick_img(*keys):
        if isinstance(imgs, dict):
            for k in keys:
                if k in imgs:
                    im = ensure_hwc_uint8(imgs[k])
                    return im
        return top

    image = {
        "cam_high": pick_img("cam_high", "high", "top"),
        "cam_low": pick_img("cam_low", "low"),
        "cam_left_wrist": pick_img("cam_left_wrist", "left_wrist", "wrist_left"),
        "cam_right_wrist": pick_img("cam_right_wrist", "right_wrist", "wrist_right"),
    }

    # ✅ 关键：key 必须叫 "image"
    payload = {
        "images": image,
        "state": state,
        "prompt": prompt,
    }
    return payload

In [19]:
def decode_action_for_env(action_vec: np.ndarray, env_action_dim: int) -> np.ndarray:
    """
    server 可能回 (H,14) 也可能回 (H,32)。
    先用“可跑”的保守策略：
      - 多了就截断
      - 少了就补零
    后面如果你确认 server 输出一定是 14，就可以删掉这段。
    """
    a = np.asarray(action_vec, dtype=np.float32).reshape(-1)
    if a.shape[0] > env_action_dim:
        a = a[:env_action_dim]
    elif a.shape[0] < env_action_dim:
        a = np.pad(a, (0, env_action_dim - a.shape[0]))
    return a


In [20]:


def main(
    env_id="gym_aloha/AlohaInsertion-v0",
    host="127.0.0.1",
    port=8000,
    prompt="Transfer cube",
    max_episode_steps=200,
    max_total_steps=400,
    replan_every=10,
    video_dir="videos_aloha_pi0",
    seed=0,
):
    os.makedirs(video_dir, exist_ok=True)

    # ✅ 覆盖默认 50 step 的 TimeLimit
    base = gym.make(env_id, render_mode="rgb_array")
    env = TimeLimit(base.unwrapped, max_episode_steps=max_episode_steps)

    env = RecordVideo(
        env,
        video_folder=video_dir,
        name_prefix="aloha_pi0",
        episode_trigger=lambda ep: True,
    )

    client = websocket_client_policy.WebsocketClientPolicy(host=host, port=port)

    obs, info = env.reset(seed=seed)

    action_chunk = None
    chunk_i = 0

    t0 = time.time()

    for t in range(max_total_steps):
        if (action_chunk is None) or (t % replan_every == 0) or (chunk_i >= len(action_chunk)):
            payload = build_aloha_sim_payload(env, obs, prompt=prompt)
            # 你可以打印 state 维度确认一下
            if t == 0:
                print("state dim:", payload["state"].shape)
                for k, im in payload["images"].items():
                    print(k, im.shape, im.dtype)

            out = client.infer(payload)
            action_chunk = np.asarray(out["actions"], dtype=np.float32)
            chunk_i = 0

            print(f"[replan] chunk shape={action_chunk.shape} at step={t}")

        a = decode_action_for_env(action_chunk[chunk_i], env.action_space.shape[0])
        chunk_i += 1

        a = np.clip(a, env.action_space.low, env.action_space.high)

        obs, reward, terminated, truncated, info = env.step(a)

        if terminated or truncated:
            print(f"[done] t={t+1} terminated={terminated} truncated={truncated} "
                  f"reward={reward} is_success={info.get('is_success', None)}")
            break

    env.close()
    print(f"[time] total_sec={time.time()-t0:.2f}")


In [23]:
if __name__ == "__main__":
    main()

state dim: (14,)
cam_high (480, 640, 3) uint8
cam_low (480, 640, 3) uint8
cam_left_wrist (480, 640, 3) uint8
cam_right_wrist (480, 640, 3) uint8


RuntimeError: Error in inference server:
Traceback (most recent call last):
  File "/workspace/projects/openpi/.venv/lib/python3.11/site-packages/PIL/Image.py", line 3299, in fromarray
    mode, rawmode = _fromarray_typemap[typekey]
                    ~~~~~~~~~~~~~~~~~~^^^^^^^^^
KeyError: ((1, 1, 480), '|u1')

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/workspace/projects/openpi/src/openpi/serving/websocket_policy_server.py", line 61, in _handler
    action = self._policy.infer(obs)
             ^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspace/projects/openpi/src/openpi/policies/policy.py", line 71, in infer
    inputs = self._input_transform(inputs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspace/projects/openpi/src/openpi/transforms.py", line 70, in __call__
    data = transform(data)
           ^^^^^^^^^^^^^^^
  File "/workspace/projects/openpi/src/openpi/transforms.py", line 190, in __call__
    data["im

In [10]:
import numpy as np
import gymnasium as gym
import gym_aloha  # trigger registration
from openpi_client import websocket_client_policy
from openpi.policies import aloha_policy

env = gym.make("gym_aloha/AlohaInsertion-v0", render_mode="rgb_array")
dm_obs = env.unwrapped._env._task.get_observation(env.unwrapped._env._physics)
print(dm_obs.keys())
for k, v in dm_obs.items():
    print(k, type(v))


odict_keys(['qpos', 'qvel', 'env_state', 'images'])
qpos <class 'numpy.ndarray'>
qvel <class 'numpy.ndarray'>
env_state <class 'numpy.ndarray'>
images <class 'dict'>
